# 0. The Obligatory Part

In [ ]:
# Run this cell if data in Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install ortools
!pip3 install ortools

In [ ]:
# Import library
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import math

# 1. Define the Data Structure

In [ ]:
# Data path
new_employee_path = '/content/drive/MyDrive/Tadika Mesra Bunga Matahari/#1 Optimization Problem/project1_task-assignment/data/fixed/fixed_data_employee.csv'
new_task_path = '/content/drive/MyDrive/Tadika Mesra Bunga Matahari/#1 Optimization Problem/project1_task-assignment/data/fixed/fixed_data_task.csv'

## 1.1. Pre-Processing: Employee Data

In [ ]:
# Read data
employee_skills_df = pd.read_csv(new_employee_path, index_col='employee_id')
employee_skills_df.drop(columns=['no', 'Role'], inplace=True, errors='ignore')

employees = employee_skills_df.index.tolist()
skills_name = employee_skills_df.columns[1:].tolist()

employee_skills_df

## 1.2. Pre-Processing: Task Data

In [ ]:
task_df = pd.read_csv(new_task_path, index_col='task_id')

tasks = task_df.index.tolist()
company_names = list(set(task_df['project_id']))
story_points = task_df['story_points'].to_dict()

task_df

In [ ]:
# convert to dictionary each company and its task
company_tasks = {}

for company in company_names:
  company_tasks[company] = task_df[task_df['project_id'] == company].index.tolist()

# sort the company tasks from C1 to C5
company_tasks = dict(sorted(company_tasks.items()))

company_tasks_df = pd.DataFrame.from_dict(company_tasks, orient='index')
company_tasks_df

## 1.3. Pre-Processing: Calculate The Similarity Error

In [ ]:
task_skills_df = task_df.drop(columns=['project_id', 'story_points'])
task_skills_df.head()

### 1.3.1 Weighted Euclidean Method

**Weight Formula**

$$
w_i = \frac{1}{1 + a \cdot max(0, v_{1i} - v_{2i})}
$$

Where:

* $w_i$ is the weight for the *i*-th element.
* $v_{1i}$ is the *i*-th element of the first vector.
* $v_{2i}$ is the *i*-th element of the second vector.
* $a$ is a weighting parameter

<br>
<br>

**Weighted Euclidean Distance Formula**

$$
d(v_1, v_2) = \sqrt{\sum_{i=1}^n w_i (v_{1i} - v_{2i})^2}
$$

Where:
* $d(v_1, v_2)$ is the weighted Euclidean distance between vectors $v_1$ and $v_2$.
* $w_i$ is the weight for the *i*-th element.
* $v_{1i}$ is the *i*-th element of the first vector.
* $v_{2i}$ is the *i*-th element of the second vector.
* $n$ is the dimensionality of the vectors.

In [ ]:
def custom_we(v1, v2, a=0.5):
  # calculate differences
  diff = v1 - v2

  # adjust weight: over qualified only
  w = 1 / (1 + a * np.maximum(0, diff))

  return w

def euclidean_similarity(emp, task):
  sum = 0
  for index, metric in enumerate(emp):
    if task[index] > 0: # if the skill criteria is not 0
      w = custom_we(emp[index], task[index]) # create weight
      sum += w * ((emp[index] - task[index])**2)
    elif task[index] == 0: # else it is, then we don't take account of this aspect
      sum += 0

  return math.sqrt(sum)


In [ ]:
# Calculate the eucliean similarity
euclidean_similarity_score = {}
count_no_match = 0

for i in tasks:
  task_skills = task_skills_df.loc[i]

  for j in employees:
    employee_skills = employee_skills_df.loc[j]

    # Filter skills to consider only those present in both project requirements and employee skills
    common_skills = [skill for skill in employee_skills.index if skill in task_skills.index]

    # check if there's at least one skill matching
    if common_skills:
      # calculate weighted euclidean distance for common skills
      euclidean_similarity_score[(i, j)] = euclidean_similarity(employee_skills[common_skills], task_skills[common_skills]) # compute euclidean distance
      euclidean_similarity_score[(i, j)] = 1 / (1 + euclidean_similarity_score[(i, j)])
    else:
      count_no_match += 1

print(count_no_match)

euclidean_similarity_score_df = pd.DataFrame.from_dict(euclidean_similarity_score, orient='index')
euclidean_similarity_score_df

# 2. Construct the Model

In [ ]:
model = cp_model.CpModel()

# 3. Build the Decision Variable

We have 3 sets
$$
sets=\begin{cases}I\:&,\:set\:of\:task\\ J&,\:set\:of\:employee\\ K&,\:set\:of\:project\end{cases}
$$

then we have parameters, scalars, and data structures. so let:
$$
i=\:task \:i \\
j=employee \:j\\
k=projects \:k\\
s_i=story\:points\:of\:task \:i \\
e_{ij}=similarity\:skills\:of\:employee\:j\:for\:task\:i \\
$$



Decision Variable

$$
x_{ijk}=Binary\:variable\:indicating\:whether\:employee\:j\:is\:assigned\:to\:task\:k\:for\:day\:i
$$
$$
y_{jk}=Binary\:variable\:indicating\:whether\:employee\:j\:is\:assigned\:to\:any\:task\:from\:company\:k
$$

In [ ]:
max_employee_workload = 20

In [ ]:
# Create decision variables for x and y
x = {}
for k, task in company_tasks.items():
    for i in task:
        for j in employees:
            x[(i, j, k)] = model.NewIntVar(0, 1, f'x_{i}_{j}_{k}')

# decision variable y represent cardinility of each employee and company
y = {}
for j in employees:
    for k in company_tasks.keys():
        y[(j, k)] = model.NewIntVar(0, 1, f'y_{j}_{k}')

# decision variables max_workload
max_workload = model.NewIntVar(0, max_employee_workload, 'max_workload')

In [ ]:
print(x)
print(y)

# 4. Subject to the Constraint

## Constraint 1: Each task is assigned to one employee
$$
\sum _{j\in J}\:x_{ijk}\:=\:1 \quad \forall i \in k, j \in J, k \in K
$$

In [ ]:
# constraint 1: each task assigned to one talent
for k, task in company_tasks.items():
    for i in task:
        model.Add(sum(x[(i, j, k)] for j in employees) == 1) 

## Constraint 2: Each employee works for one company at a time
$$
Pre-Constraint\:2:\sum _{i\in I_k}x_{ijk}>0\:\rightarrow \:y_{jk}=1\:\forall j\in J,\:\forall k\in K\:
$$

In [ ]:
# pre-processing constraint 2
for j in employees:
    for k, task in company_tasks.items():
        # Create a temporary list to hold the sum of x[i][j][k] for all i
        temp_sum = []
        for i in task:
            temp_sum.append(x[(i, j, k)])
        # Add a constraint to the model: y[j][k] is 1 if the sum of x[i][j][k] for all i is > 0, and 0 otherwise
        model.Add(sum(temp_sum) > 0).OnlyEnforceIf(y[(j, k)])
        model.Add(sum(temp_sum) <= 0).OnlyEnforceIf(y[(j, k)].Not())

$$
Constraint\:2:\sum _{k\in K}y_{jk}\le 1\:\forall j\in J,k\in K\:
$$

In [ ]:
# create constraint 2: each employee can only work on one task
for j in employees:
    # The sum of y[j][k] for all companies (k) should be <= 1
    model.Add(sum(y[(j, k)] for k in company_tasks.keys()) <= 1)

## Constraint 3: Employee workload doesn't exceed the capacity
$$
Constraint\:3:\:\sum _{j\in J}s_i\cdot x_{ijk}\le 20\:\forall i\in k,j\in J, k\in K\:
$$

In [ ]:
# constraint 3: employee workload doesn't exceed the capacity
for j in employees:
  model.Add(sum(story_points[i] * x[i, j, k] for k, tasks in company_tasks.items() for i in tasks) <= max_employee_workload)

## Constraint 4: Maximum workload is greater than or equal to the workload of each employee
$$
Constraint\:4:\:maxworkload\ge \sum _{i,\:k}s_i\cdot x_{i,j,k},\:\forall j\in J\:\:
$$

In [ ]:
# constraint 4: max_workload is greater than or equal to the workload of each employee
for j in employees:
    model.Add(max_workload >= sum(story_points[i] * x[i, j, k] for k, tasks in company_tasks.items() for i in tasks))

# 5. Single Objective 1: Minimize the Idle Employee

## 5.1. Set the Objective Model

## Single Objective Minimize Idle Employee
$$
min.\:I_j=\sum _{j\in \:J}\:\left(1\:-\:\sum _{k\in \:K}\:y_{jk}\right)\quad \: \forall j\in J, k\in K\quad \tag{1}
$$

In [ ]:
# objective 1
I = []

for j in employees:
  obj1 = 1 - sum(y[j, k] for k in company_tasks.keys())
  I.append(obj1)

I_total_idle_employee = sum(I)

# single objective 1
model.Minimize(I_total_idle_employee)

## 5.2. Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Idle Employees: {solver.Value(I_total_idle_employee)}')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {euclidean_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(euclidean_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

## 5.3. Print the Result

In [ ]:
# Convert dictionary to DataFrame
result1 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result1.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result1

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# make boxplot for objective 1 variable from the similarity score
similarity_score1 = result1['similarity_score'].explode().reset_index(drop=True)
similarity_score1.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# 6. Single Objective 2: Maximize the Similarity Error

## 6.1. Set the Objective Model

## Single Objective Maximize Similarity Score
$$
max.\:E_{ij}=\sum_{i\in I}\:\sum _{j\in J}\:e_{ij}\cdot x_{ijk}\quad \:\:\forall \:i\in k, \:j \in J, \:k \in K\quad\tag{2}
$$

In [ ]:
# objective 2
# E_total_similarity_score = 0

for k, tasks in company_tasks.items():
  E_total_similarity_score = sum(euclidean_similarity_score[i, j] * x[i, j, k] for i in tasks for j in employees)


model.Minimize(E_total_similarity_score)


# E_total_similarity_score = []

# for k, tasks in company_tasks.items():
#   for j in employees:
#     obj2 = sum(euclidean_similarity_score[i, j] * x[i, j, k] for i in tasks for j in employees)
#     E_total_similarity_score.append(obj2)

# model_2.Maximize(E_total_similarity_score)

## 6.2. Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Similarity Score: {solver.Value(E_total_similarity_score)}\n')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {euclidean_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(euclidean_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

## 6.3. Print the Result

In [ ]:
# Convert dictionary to DataFrame
result2 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result2.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result2

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# make boxplot for objective 2 variable from the similarity score
similarity_score2 = result2['similarity_score'].explode().reset_index(drop=True)
similarity_score2.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# 7. Single Objective 3: Balancing Workload

## 7.1 Set The Objective Model

## Single Objective Balancing Workload Employee

In [ ]:
model.Minimize(max_workload)

## 7.2 Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

## 7.3 Print the Result

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Similarity Score: {solver.Value(E_total_similarity_score)}\n')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {euclidean_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(euclidean_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

In [ ]:
# Convert dictionary to DataFrame
result3 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result3.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result3

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# make boxplot for objective 3 variable from the similarity score
similarity_score3 = result3['similarity_score'].explode().reset_index(drop=True)
similarity_score3.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# Boxplot Visualization 3 Single Objective

In [ ]:
# merge all boxplot in one graph
plt.figure(figsize=(10, 5))
plt.boxplot([similarity_score1, similarity_score2, similarity_score3], labels=['Objective 1', 'Objective 2', 'Objective 3'])
plt.title('Similarity Score Boxplot')
plt.show()

# 8. Multi-Objective Approach

## 8.1. Set The Objective Model

## Multi Objective Approach

$$
min.\:W=\alpha \cdot min.\:I_j+\beta \cdot max.\:E_{ij} \quad \tag{3}
$$

In [ ]:
beta = 0.7
theta = 0.3

In [ ]:
# objective 1
# I = []

# for j in employees:
#   obj1 = 1 - sum(y[j, k] for k in company_tasks.keys())
#   I.append(obj1)

# I_total_idle_employee = sum(I)

# # objective 2
# for k, tasks in company_tasks.items():
#   E_total_similarity_score = sum(euclidean_similarity_score[i, j] * x[i, j, k] for i in tasks for j in employees)

model.Minimize((beta * E_total_similarity_score) + (theta * max_workload))

## 8.2. Solve the Model

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution Found!')
  print(f'Objective Value: {solver.ObjectiveValue()}\n')

  print(f'Total Idle Employees: {solver.Value(I_total_idle_employee)}')
  print(f'Total Similarity Score: {solver.Value(E_total_similarity_score)}\n')

  x_hat = {}

  for j in employees:
    task = []
    sim = []
    sp = 0
    wasted_sp = 0
    comp = []

    for k, tasks in company_tasks.items():
      for i in tasks:
        if solver.Value(x[i, j, k]) == 1:
          print(f'Task {i} assigned to Employee {j}')
          print(f'Company\t\t\t: {k}')
          print(f'Story Points\t\t: {story_points[i]}')
          print(f"Similarity score\t: {euclidean_similarity_score[i, j]:.10f}\n")

          task.append(i)
          sim.append(euclidean_similarity_score[i, j])
          comp.append(k)
          sp += story_points[i]

    if sp > 0:
      wasted_sp = max_employee_workload - sp
      x_hat[j] = comp, task, sp, wasted_sp, sim

else:
  print('No Solution Found!')
  x_hat = {}

## 8.3. Print the Result

In [ ]:
# Convert dictionary to DataFrame
result4 = pd.DataFrame([(key, value[0], value[1], value[2], value[3], value[4]) for key, value in x_hat.items()],
                      columns=['employee', 'company', 'assigned_task', 'sum_sp', 'wasted_sp', 'similarity_score'])

# Set 'company' as index
result4.set_index('employee', inplace=True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

result4

In [ ]:
total_employee = len(employees)
total_sp = sum(story_points.values())
total_active_employee = len(set(employee for employee in x_hat.keys()))
total_active_sp = sum(value[2] for value in x_hat.values())
total_idle_employee = total_employee - total_active_employee
total_wasted_sp = total_sp - total_active_sp

print(f'Total Employee\t\t\t: {total_employee}')
print(f'Total Active Employee\t\t: {total_active_employee}\t{(total_active_employee/total_employee)*100:.2f}%')
print(f'Total Idle Employee\t\t: {total_idle_employee}\t{(total_idle_employee/total_employee)*100:.2f}%\n')
print(f'Total Story Points\t\t: {total_sp}')
print(f'Total Active Story Points\t: {total_active_sp}\t{(total_active_sp/total_sp)*100:.2f}%')
print(f'Total Wasted Story Points\t: {total_wasted_sp}\t{(total_wasted_sp/total_sp)*100:.2f}%\n')

In [ ]:
# Data for employees
employee_labels = ['Total Employees', 'Total Active Employees', 'Total Idle Employees']
employee_values = [total_employee, total_active_employee, total_idle_employee]

# Data for story points
sp_labels = ['Total Story Points', 'Total Active Story Points', 'Total Wasted Story Points']
sp_values = [total_sp, total_active_sp, total_wasted_sp]

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot for employees
ax1.barh(employee_labels, employee_values, color='skyblue')
ax1.set_xlabel('Count')
ax1.set_title('Summary of Employees')
ax1.grid(axis='x', linestyle='--', alpha=0.7)

# Plot for story points
ax2.barh(sp_labels, sp_values, color='lightgreen')
ax2.set_xlabel('Count')
ax2.set_title('Summary of Story Points')
ax2.grid(axis='x', linestyle='--', alpha=0.7)

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# make boxplot for multi objective variable from the similarity score
similarity_score_multi = result4['similarity_score'].explode().reset_index(drop=True)
similarity_score_multi.plot(kind='box')
plt.title('Similarity Score Boxplot')
plt.show()

# Boxplot Visualization

In [ ]:
# merge all boxplot in one graph
plt.figure(figsize=(10, 5))
plt.boxplot([similarity_score1, similarity_score2, similarity_score3, similarity_score_multi],
            labels=['Obj 1: Min Idle Emp', 'Obj 2: Max Similarity Score', 'Obj 3: Balancing the Workload', 'Multi Objective'])
plt.title('Similarity Score Boxplot')
plt.xticks(rotation=15)
plt.show()

# 9. Strategic Analysis